# Customers Orders Case study - II

### Syed Khaleelullah

### For each year we need the following information:

1. Total revenue for the current year
2. New Customer Revenue e.g., new customers not present in previous year only
3. Existing Customer Growth. To calculate this, use the Revenue of existing customers for current year –(minus) Revenue of existing customers from the previous year
4. Revenue lost from attrition
5. Existing Customer Revenue Current Year
6. Existing Customer Revenue Prior Year
7. Total Customers Current Year
8. Total Customers Previous Year
9. New Customers
10. Lost Customers

In [1]:
#Let's import the libraries
import sqlite3
import pandas as pd

In [2]:
conn = sqlite3.connect('test.db')
cursor = conn.cursor()

cursor.execute('''DROP TABLE CASESTUDY''')

cursor.execute('''CREATE TABLE CASESTUDY (customer_email CHAR(255), net_revenue FLOAT, YEAR INTEGER)''')

conn.commit()

In [3]:
# read csv to dataframe and then to CUSTOMER table in database
read_data = pd.read_csv('casestudy.csv', usecols=['customer_email','net_revenue','year'])
read_data.to_sql('CASESTUDY', conn, if_exists='append', index = False)
conn.commit()

1. Total Revenue for the current year

In [4]:
# total revenue for current year (2017)
cursor.execute('''
SELECT round(sum(net_revenue), 2) FROM CASESTUDY WHERE year = 2017
''')

print("Total Revenue for 2017: $",cursor.fetchall())

Total Revenue for 2017: $ [(31417495.03,)]


2. New Customer Revenue e.g., new customers not present in previous year only

In [5]:
# Total new Customer Revenue e.g. new customer revenue from 2017 only
cursor.execute('''
SELECT round(sum(net_revenue),2) FROM CASESTUDY 
WHERE year = 2017 AND customer_email NOT IN( SELECT DISTINCT customer_email FROM CASESTUDY WHERE year < 2017)
''')

print("New Customer Revenue 2017: $", cursor.fetchall())

New Customer Revenue 2017: $ [(28676607.64,)]


3. Existing Customer Growth. To calculate this, use the Revenue of existing customers for current year –(minus) Revenue of existing customers from the previous year

In [6]:
# Revenue of existing customers for current year i.e.customers exisiting prior to 2017 and revenue from 2017
cursor.execute('''
SELECT round(sum(net_revenue),2) FROM CASESTUDY
WHERE year = 2017 AND customer_email IN( SELECT DISTINCT customer_email FROM CASESTUDY WHERE year < 2017)
''')

print("Existing Customer Revenue 2017: $", cursor.fetchall())

Existing Customer Revenue 2017: $ [(2740887.39,)]


4. Revenue lost from attrition

In [7]:
# Revenue lost from attrition e.g. revenue lost between 2016 and 2017
cursor.execute('''
SELECT (SELECT round(sum(net_revenue),2) FROM CASESTUDY WHERE year = 2017) - (SELECT round(sum(net_revenue),2)
FROM CASESTUDY  WHERE year = 2016) 
''')

print("Total Revenue lost from attrition between 2016 and 2017): $", cursor.fetchall())

Total Revenue lost in attrition between 2016 and 2017): $ [(5686551.440000001,)]


5. Existing Customer Revenue Current Year

In [8]:
# Existing Customer Revenue Current Year i.e.customer email from 2017 and revenue from 2017
cursor.execute('''
SELECT round(sum(net_revenue),2) FROM CASESTUDY
WHERE year = 2017 AND customer_email IN(SELECT DISTINCT customer_email FROM CASESTUDY as C WHERE year = 2017)
''')

print("Existing Customer Revenue for current year = 2017:", cursor.fetchall())

Revenue for 2017 from exsiting customers: [(31417495.03,)]


6. Existing Customer Revenue Prior Year

In [9]:
# Existing Customer Revenue Prior Year i.e. current customer email from 2017 but revenue for 2016
cursor.execute('''
SELECT round(sum(net_revenue),2) FROM CASESTUDY
WHERE year = 2016 AND customer_email IN(SELECT DISTINCT customer_email FROM CASESTUDY WHERE year = 2017)
''')

print("Existing Customer Revenue for previous year = 2016:", cursor.fetchall())

Revenue for 2016 from exsiting customers: [(2620648.65,)]


7. Total Customers Current Year

In [10]:
# total number of customers in current year i.e. 2017
cursor.execute('''
SELECT count(DISTINCT customer_email) FROM CASESTUDY
WHERE year = 2017
''')

print("Total Customers in current year = 2017: ", cursor.fetchall())

Total Customers in 2017:  [(249987,)]


8. Total Customers Previous Year

In [11]:
# total number of customers in previous year i.e. 2016
cursor.execute('''
SELECT count(DISTINCT customer_email) FROM CASESTUDY
WHERE year = 2016
''')

print("Total Customers in previous year = 2016: ", cursor.fetchall())

Total Customers in 2016:  [(204646,)]


9. New Customers

In [12]:
# new total customers in 2017 only
cursor.execute('''
SELECT count(DISTINCT customer_email) FROM CASESTUDY 
WHERE year = 2017 and customer_email NOT IN(SELECT DISTINCT customer_email FROM CASESTUDY WHERE year < 2017)
''')

print("New Customers in 2017: ", cursor.fetchall())

New Customers in 2017 only:  [(228262,)]


10. Lost Customers

In [13]:
# lost customers i.e. customers in previous years but not in 2017
cursor.execute('''
SELECT count(DISTINCT customer_email) FROM CASESTUDY
WHERE year < 2017 AND customer_email NOT IN (SELECT DISTINCT customer_email FROM CASESTUDY WHERE year = 2017)
''')

print("Customers lost in 2017: ", cursor.fetchall())
conn.close()

Customers lost in 2017:  [(354631,)]
